In [3]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

In [4]:
from pyspark.sql import functions as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from umap import UMAP
from sklearn.decomposition import PCA

/home/acmiy/birdclef-2023/venv/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from birdclef.utils import get_spark

spark = get_spark(memory="2g")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/16 05:22:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
df = spark.read.parquet(
    "../data/processed/birdclef-2022/birdnet-embeddings-with-neighbors/v1"
)
df.printSchema()
df.show(n=1, vertical=True)

root
 |-- id: integer (nullable = true)
 |-- filename: string (nullable = true)
 |-- start_sec: double (nullable = true)
 |-- end_sec: double (nullable = true)
 |-- confidence: double (nullable = true)
 |-- birdnet_label: string (nullable = true)
 |-- birdnet_common_name: string (nullable = true)
 |-- emb: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- primary_label: string (nullable = true)
 |-- secondary_labels: string (nullable = true)
 |-- type: string (nullable = true)
 |-- neighbors: array (nullable = true)
 |    |-- element: long (containsNull = true)
 |-- distances: array (nullable = true)
 |    |-- element: double (containsNull = true)



-RECORD 0-----------------------------------
 id                  | 11876                
 filename            | belkin1/XC321817.ogg 
 start_sec           | 15.0                 
 end_sec             | 18.0                 
 confidence          | 0.1058               
 birdnet_label       | rucspa               
 birdnet_common_name | Rufous-crowned Sp... 
 emb                 | [0.54678416, 0.58... 
 primary_label       | belkin1              
 secondary_labels    | []                   
 type                | ['song']             
 neighbors           | [11877, 11876, 11... 
 distances           | [0.0, 0.0, 0.0, 4... 
only showing top 1 row



* https://ebird.org/species/brnowl
* https://ebird.org/species/mallar3

In [7]:
df.groupBy("primary_label").count().orderBy(F.desc("count")).show(n=10)

+-------------+-----+
|primary_label|count|
+-------------+-----+
|       normoc|25068|
|       houspa|18173|
|       skylar|12849|
|       norcar|12216|
|       wesmea| 9479|
|       brnowl| 8186|
|      mallar3| 7489|
|       houfin| 6616|
|       commyn| 6524|
|       dunlin| 6069|
+-------------+-----+
only showing top 10 rows



In [8]:
subset_species = ["normoc", "houspa", "skylar"]
subset = df.where(F.col("primary_label").isin(subset_species))
subset.groupBy("primary_label", "birdnet_label").count().orderBy(F.desc("count")).show()

+-------------+-------------+-----+
|primary_label|birdnet_label|count|
+-------------+-------------+-----+
|       houspa|       houspa| 8683|
|       skylar|       skylar| 6816|
|       normoc|       normoc| 5742|
|       houspa|      itaspa1| 3005|
|       normoc|       tromoc| 1703|
|       normoc|       crithr| 1598|
|       skylar|      callar1| 1273|
|       skylar|      orisky1| 1106|
|       houspa|       eutspa| 1084|
|       normoc|       cubthr| 1079|
|       houspa|      spaspa1|  764|
|       normoc|      patmoc1|  470|
|       normoc|       logshr|  449|
|       normoc|       calthr|  375|
|       normoc|       cacwre|  335|
|       normoc|       brnthr|  328|
|       houspa|      sposta1|  307|
|       normoc|       scoori|  302|
|       houspa|       eucdov|  289|
|       normoc|      chbmoc1|  277|
+-------------+-------------+-----+
only showing top 20 rows



In [10]:
subset.toPandas().to_parquet("../data/processed/2022-01-15-assessment.parquet")

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/home/acmiy/birdclef-2023/venv/lib/python3.7/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/lib/python3.7/socket.py", line 589, in readinto
    return self._sock.recv_into(b)
ConnectionResetError: [Errno 104] Connection reset by peer

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/acmiy/birdclef-2023/venv/lib/python3.7/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/acmiy/birdclef-2023/venv/lib/python3.7/site-packages/py4j/clientserver.py", line 540, in send_command
    "Error while sending or receiving", e, proto.ERROR_ON_RECEIVE)
py4j.protocol.Py4JNetworkError: Error while sending or receiving
